In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [191]:
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
import yfinance as yf

import ipywidgets as ipw
from IPython.display import HTML, display,Javascript,clear_output
from ipywidgets import interact, interactive, fixed, interact_manual
# https://www.youtube.com/watch?v=abXuRM0QVD0&ab_channel=Elliott


def plot_chart(stock_df):
    import plotly.express as px
    fig = go.Figure()
    fig.add_trace(go.Candlestick(x=stock_df.index,
                    open=stock_df['Open'],
                    high=stock_df['High'],
                    low=stock_df['Low'],
                    close=stock_df['Close'],name='Candlestick'))

    #fig = px.line(google_df, x=google_df.index, y="30moving average")

    fig.add_trace(go.Scatter(x=stock_df.index, y=stock_df["150 days moving average"],
                        line=dict(color='yellow', width=4),
                        name='150 moving average'))
    
    fig.add_trace(go.Scatter(x=stock_df.index, y=stock_df["50 days moving average"],
                        line=dict(color='blue', width=4),
                        name='50 moving average'))

    fig.layout.yaxis=dict(
           autorange = True,
           fixedrange= False
       )


    fig.update_layout(
        title=stock_df.name+" Stock Price",
        title_x=0.5,
        xaxis_title="Date",
        yaxis_title="Stock Price",
        legend_title="Legend",
        font=dict(
            family="Courier New, monospace",
            size=12,
            color="RebeccaPurple"
        )
    )

    fig.update_layout(legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ))

    fig.update_xaxes(title_font_family="Arial")
    fig.show()
    
def linechart(stock_df):
    fig = go.Figure([go.Scatter(x=stock_df.index, y=stock_df['High'])])
    fig.update_layout(
        title=stock_df.name+" Stock Price",
        title_x=0.5,
        xaxis_title="Date",
        yaxis_title="Stock Price",
        legend_title="Legend",
        font=dict(
            family="Courier New, monospace",
            size=12,
            color="RebeccaPurple"
        )
    )
    fig.show()


def findstable(df,date,last,percentage=0.05):
    df=df.loc[date:]
    start=date
    price=df.loc[date,'Open']
    interval=[price*(1-percentage),price*(1+percentage)]
    list1=[]
    day_count=0
    
    for date,row in df.iterrows():
        if (interval[0]<=row['High']<=interval[1]) and (interval[0]<=row['Low']<=interval[1]) and date!=df.index[-1]:
            day_count+=1
        else:
            if day_count>=last or date==df.index[-1]:
                list1.append((price,start,date,interval,day_count))
            start=date    
            price=row['Open']
            interval=[price*(1-percentage),price*(1+percentage)]
            day_count=0
    return pd.DataFrame(list1,columns=['price','Start_date','End_date',str(percentage*100)+'% Interval','day count'])


        
def selectStock(df,Stock_Search,date):
    date=date.value.strftime("%Y-%m-%d")
    temp=df.iloc[:, df.columns.get_level_values(1)==Stock_Search.value].copy()
    temp=temp.loc[date:]
    temp=temp.stack().droplevel(1)
    temp.name=Stock_Search.label
    plot_chart(temp)
    #linechart(stock_df)    
    df=findstable(temp,date,15)
    del temp
    display(df)
    
date=ipw.DatePicker(
    description='Start Date',
    disabled=False,
    value=datetime(year=2020,month=1,day=3)
)

Dropdown=ipw.Dropdown( 
    options=[('Google','GOOG'),('Microsoft Corporation','MSFT'),('Meituan','MPNGF')],description='Stock:') #=[('One', 1), ('Two', 2), ('Three', 3)],

btn=ipw.Button(description='Search',
           disabled=False,
           buttion_style='',
           tooltip='Click me'
)


stock_df=yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "goog msft mpngf",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "max")

stock_list=stock_df.columns.get_level_values(1).unique()
stock_df[[('50 days moving average',stock) for stock in stock_list]]=stock_df.xs('Close', axis=1, level=0).rolling(window=50).mean()
stock_df[[('150 days moving average',stock) for stock in stock_list]]=stock_df.xs('Close', axis=1, level=0).rolling(window=150).mean()


search_row=ipw.HBox([Dropdown,date,btn])

def is_onclick(b):
    clear_output(True)
    display(search_row)
    selectStock(stock_df,Dropdown,date)
    
btn.on_click(is_onclick)
btn.click()

,price,Start_date,End_date,5.0% Interval,day count
0,1119.989990,2019-06-20,2019-07-26,"[1063.9904907226562, 1175.9894897460938]",24
1,1170.040039,2019-08-05,2019-09-12,"[1111.538037109375, 1228.542041015625]",26
2,1275.449951,2019-10-28,2019-12-06,"[1211.6774536132812, 1339.2224487304688]",27
3,1333.439941,2019-12-06,2020-01-07,"[1266.7679443359375, 1400.1119384765625]",19
4,1462.910034,2020-01-17,2020-02-25,"[1389.764532470703, 1536.055535888672]",24
5,1378.280029,2020-05-11,2020-06-08,"[1309.3660278320312, 1447.1940307617188]",18
6,1498.930054,2020-07-24,2020-08-20,"[1423.9835510253906, 1573.8765563964844]",18
7,1790.900024,2020-11-09,2020-11-30,"[1701.3550231933593, 1880.4450256347657]",13


In [24]:
def calculate_change(x,y):
    return (x-y)/x

In [39]:
tickers =yf.Ticker('goog msft mpngf')

In [168]:
data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "goog msft mpngf",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "max")



[*********************100%***********************]  3 of 3 completed


(8752, 3)

In [178]:
stock_df

Adj Close                               Close             \
                   GOOG      MPNGF        MSFT         GOOG      MPNGF   
Date                                                                     
1986-03-13          NaN        NaN    0.061893          NaN        NaN   
1986-03-14          NaN        NaN    0.064103          NaN        NaN   
1986-03-17          NaN        NaN    0.065209          NaN        NaN   
1986-03-18          NaN        NaN    0.063551          NaN        NaN   
1986-03-19          NaN        NaN    0.062446          NaN        NaN   
...                 ...        ...         ...          ...        ...   
2020-11-23  1734.859985  39.299999  210.110001  1734.859985  39.299999   
2020-11-24  1768.880005  39.250000  213.860001  1768.880005  39.250000   
2020-11-25  1771.430054  38.154999  213.869995  1771.430054  38.154999   
2020-11-27  1793.189941  40.480000  215.229996  1793.189941  40.480000   
2020-11-30  1760.739990  38.945999  214.070007  1760.739990  38.945999   

                               High                                 Low  ...  \
                  MSFT         GOOG      MPNGF        MSFT         GOOG  ...   
Date                                                                     ...   
1986-03-13    0.097222          NaN        NaN    0.101562          NaN  ...   
1986-03-14    0.100694          NaN        NaN    0.102431          NaN  ...   
1986-03-17    0.102431          NaN        NaN    0.103299          NaN  ...   
1986-03-18    0.099826          NaN        NaN    0.103299          NaN  ...   
1986-03-19    0.098090          NaN        NaN    0.100694          NaN  ...   
...                ...          ...        ...         ...          ...  ...   
2020-11-23  210.110001  1753.900024  41.060001  212.289993  1717.719971  ...   
2020-11-24  213.860001  1771.599976  39.950001  214.250000  1727.689941  ...   
2020-11-25  213.869995  1778.540039  39.169998  215.289993  1756.540039  ...   
2020-11-27  215.229996  1804.000000  40.799999  216.270004  1772.439941  ...   
2020-11-30  214.070007  1788.064941  39.299999  214.759995  1755.000000  ...   

                  Open     Volume                      50 days moving average  \
                  MSFT       GOOG    MPNGF        MSFT                   GOOG   
Date                                                                            
1986-03-13    0.088542        NaN      NaN  1031788800                    NaN   
1986-03-14    0.097222        NaN      NaN   308160000                    NaN   
1986-03-17    0.100694        NaN      NaN   133171200                    NaN   
1986-03-18    0.102431        NaN      NaN    67766400                    NaN   
1986-03-19    0.099826        NaN      NaN    47894400                    NaN   
...                ...        ...      ...         ...                    ...   
2020-11-23  210.949997  2161600.0  12178.0    25683500            1590.186799   
2020-11-24  209.589996  1578000.0  29499.0    33979700            1594.735601   
2020-11-25  215.110001  1045800.0  17330.0    21012900            1599.746201   
2020-11-27  214.850006   884900.0   9264.0    14512200            1605.699399   
2020-11-30  214.100006  1821900.0  40055.0    33035400            1611.714399   

                                150 days moving average                         
                MPNGF      MSFT                    GOOG      MPNGF        MSFT  
Date                                                                            
1986-03-13        NaN       NaN                     NaN        NaN         NaN  
1986-03-14        NaN       NaN                     NaN        NaN         NaN  
1986-03-17        NaN       NaN                     NaN        NaN         NaN  
1986-03-18        NaN       NaN                     NaN        NaN         NaN  
1986-03-19        NaN       NaN                     NaN        NaN         NaN  
...               ...       ...                     ...        ...       

In [171]:
data.columns.get_level_values(1)

Index(['GOOG', 'MPNGF', 'MSFT', 'GOOG', 'MPNGF', 'MSFT', 'GOOG', 'MPNGF',
       'MSFT', 'GOOG', 'MPNGF', 'MSFT', 'GOOG', 'MPNGF', 'MSFT', 'GOOG',
       'MPNGF', 'MSFT'],
      dtype='object')

In [172]:
data[[('150 days moving average',stock) for stock in data.columns.get_level_values(1).unique()]]=data.xs('Close', axis=1, level=0).rolling(window=50).mean()

[('150 days moving average', 'GOOG'),
 ('150 days moving average', 'MPNGF'),
 ('150 days moving average', 'MSFT')]

In [88]:
temp.stack().index

MultiIndex([('2004-08-19', 'GOOG'),
            ('2004-08-20', 'GOOG'),
            ('2004-08-23', 'GOOG'),
            ('2004-08-24', 'GOOG'),
            ('2004-08-25', 'GOOG'),
            ('2004-08-26', 'GOOG'),
            ('2004-08-27', 'GOOG'),
            ('2004-08-30', 'GOOG'),
            ('2004-08-31', 'GOOG'),
            ('2004-09-01', 'GOOG'),
            ...
            ('2020-11-23', 'GOOG'),
            ('2020-11-23',     ''),
            ('2020-11-24', 'GOOG'),
            ('2020-11-24',     ''),
            ('2020-11-25', 'GOOG'),
            ('2020-11-25',     ''),
            ('2020-11-27', 'GOOG'),
            ('2020-11-27',     ''),
            ('2020-11-30', 'GOOG'),
            ('2020-11-30',     '')],
           names=['Date', None], length=8151)

In [102]:
selection=data.stack().index.get_level_values(1)=='GOOG'
selection

array([False, False, False, ...,  True, False, False])

In [106]:
data.stack()[selection].droplevel(1)

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2004-08-19,49.982655,49.982655,51.835709,47.800831,49.813286,44871300.0
2004-08-20,53.952770,53.952770,54.336334,50.062355,50.316402,22942800.0
2004-08-23,54.495735,54.495735,56.528118,54.321388,55.168217,18342800.0
2004-08-24,52.239193,52.239193,55.591629,51.591621,55.412300,15319700.0
2004-08-25,52.802086,52.802086,53.798351,51.746044,52.284027,9232100.0
...,...,...,...,...,...,...
2020-11-23,1734.859985,1734.859985,1753.900024,1717.719971,1749.599976,2161600.0
2020-11-24,1768.880005,1768.880005,1771.599976,1727.689941,1730.500000,1578000.0
2020-11-25,1771.430054,1771.430054,1778.540039,1756.540039,1772.890015,1045800.0


In [107]:
data

Adj Close                               Close             \
                   GOOG      MPNGF        MSFT         GOOG      MPNGF   
Date                                                                     
1986-03-13          NaN        NaN    0.061893          NaN        NaN   
1986-03-14          NaN        NaN    0.064103          NaN        NaN   
1986-03-17          NaN        NaN    0.065209          NaN        NaN   
1986-03-18          NaN        NaN    0.063551          NaN        NaN   
1986-03-19          NaN        NaN    0.062446          NaN        NaN   
...                 ...        ...         ...          ...        ...   
2020-11-23  1734.859985  39.299999  210.110001  1734.859985  39.299999   
2020-11-24  1768.880005  39.250000  213.860001  1768.880005  39.250000   
2020-11-25  1771.430054  38.154999  213.869995  1771.430054  38.154999   
2020-11-27  1793.189941  40.480000  215.229996  1793.189941  40.480000   
2020-11-30  1760.739990  38.945999  214.070007  1760.739990  38.945999   

                               High                                 Low  \
                  MSFT         GOOG      MPNGF        MSFT         GOOG   
Date                                                                      
1986-03-13    0.097222          NaN        NaN    0.101562          NaN   
1986-03-14    0.100694          NaN        NaN    0.102431          NaN   
1986-03-17    0.102431          NaN        NaN    0.103299          NaN   
1986-03-18    0.099826          NaN        NaN    0.103299          NaN   
1986-03-19    0.098090          NaN        NaN    0.100694          NaN   
...                ...          ...        ...         ...          ...   
2020-11-23  210.110001  1753.900024  41.060001  212.289993  1717.719971   
2020-11-24  213.860001  1771.599976  39.950001  214.250000  1727.689941   
2020-11-25  213.869995  1778.540039  39.169998  215.289993  1756.540039   
2020-11-27  215.229996  1804.000000  40.799999  216.270004  1772.439941   
2020-11-30  214.070007  1788.064941  39.299999  214.759995  1755.000000   

                                          Open                         \
                MPNGF        MSFT         GOOG      MPNGF        MSFT   
Date                                                                    
1986-03-13        NaN    0.088542          NaN        NaN    0.088542   
1986-03-14        NaN    0.097222          NaN        NaN    0.097222   
1986-03-17        NaN    0.100694          NaN        NaN    0.100694   
1986-03-18        NaN    0.098958          NaN        NaN    0.102431   
1986-03-19        NaN    0.097222          NaN        NaN    0.099826   
...               ...         ...          ...        ...         ...   
2020-11-23  39.189999  208.160004  1749.599976  41.060001  210.949997   
2020-11-24  38.029999  208.860001  1730.500000  38.029999  209.589996   
2020-11-25  37.750000  212.460007  1772.890015  39.169998  215.110001   
2020-11-27  39.400002  214.039993  1773.089966  39.400002  214.850006   
2020-11-30  37.904999  210.839996  1781.183960  37.904999  214.100006   

               Volume                       
                 GOOG    MPNGF        MSFT  
Date                                        
1986-03-13        NaN      NaN  1031788800  
1986-03-14        NaN      NaN   308160000  
1986-03-17        NaN      NaN   133171200  
1986-03-18        NaN      NaN    67766400  
1986-03-19        NaN      NaN    47894400  
...               ...      ...         ...  
2020-11-23  2161600.0  12178.0    25683500  
2020-11-24  1578000.0  29499.0    33979700  
2020-11-25  1045800.0  17330.0    21012900  
2020-11-27   884900.0   9264.0    14512200  
2020-11-30  1821900.0  40055.0    33035400  

[8752 rows x 18 columns]

In [108]:
data.columns

MultiIndex([('Adj Close',  'GOOG'),
            ('Adj Close', 'MPNGF'),
            ('Adj Close',  'MSFT'),
            (    'Close',  'GOOG'),
            (    'Close', 'MPNGF'),
            (    'Close',  'MSFT'),
            (     'High',  'GOOG'),
            (     'High', 'MPNGF'),
            (     'High',  'MSFT'),
            (      'Low',  'GOOG'),
            (      'Low', 'MPNGF'),
            (      'Low',  'MSFT'),
            (     'Open',  'GOOG'),
            (     'Open', 'MPNGF'),
            (     'Open',  'MSFT'),
            (   'Volume',  'GOOG'),
            (   'Volume', 'MPNGF'),
            (   'Volume',  'MSFT')],
           )